In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp submission

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import os
import uuid
import numpy as np
import pandas as pd
from typing import Union
from copy import deepcopy
from random import choices
from tqdm.auto import tqdm
from datetime import datetime
from abc import abstractmethod
from string import ascii_uppercase
from rich import print as rich_print
from numerapi import NumerAPI, SignalsAPI
from dateutil.relativedelta import relativedelta, FR

from numerblox.download import BaseIO
from numerblox.key import Key

## 0. BaseSubmitter

`BaseSubmitter` handles all submission logic common to Numerai Classic and Numerai Signals. Under the hood directory logic is handled by `BaseIO`.
Each submittor should inherit from `BaseSubmitter` and implement the `.save_csv` method.

In [ ]:
#| export
class BaseSubmitter(BaseIO):
    """
    Basic functionality for submitting to Numerai. \n
    Uses numerapi under the hood.
    More info: https://numerapi.readthedocs.io/ \n

    :param directory_path: Directory to store and read submissions from. \n
    :param api: NumerAPI or SignalsAPI
    """
    def __init__(self, directory_path: str, api: Union[NumerAPI, SignalsAPI]):
        super().__init__(directory_path)
        self.api = api

    @abstractmethod
    def save_csv(
        self,
        dataf: pd.DataFrame,
        file_name: str,
        cols: Union[str, list],
        *args,
        **kwargs,
    ):
        """
        For Numerai Classic: Save index column + 'cols' (targets) to CSV.
        For Numerai Signals: Save ticker, friday_date, data_type and signal columns to CSV.
        """
        ...

    def upload_predictions(self, file_name: str, model_name: str, *args, **kwargs):
        """
        Upload CSV file to Numerai for given model name.
        :param file_name: File name/path relative to directory_path.
        :param model_name: Lowercase raw model name (For example, 'integration_test').
        """
        full_path = str(self.dir / file_name)
        model_id = self._get_model_id(model_name=model_name)
        api_type = str(self.api.__class__.__name__)
        rich_print(
            f":airplane: {api_type}: Uploading predictions from '{full_path}' for model [bold blue]'{model_name}'[/bold blue] (model_id='{model_id}') :airplane:"
        )
        self.api.upload_predictions(
            file_path=full_path, model_id=model_id, *args, **kwargs
        )
        rich_print(
            f":thumbs_up: {api_type} submission of '{full_path}' for [bold blue]{model_name}[/bold blue] is successful! :thumbs_up:"
        )

    def full_submission(
        self,
        dataf: pd.DataFrame,
        model_name: str,
        cols: Union[str, list],
        file_name: str = 'submission.csv',
        *args,
        **kwargs,
    ):
        """
        Save DataFrame to csv and upload predictions through API.

        :param dataf: Main DataFrame containing `cols`.
        :param model_name: Lowercase Numerai model name.
        :param file_name: path to save model to relative to base directory.
        :param cols: Columns to be saved in submission file.
        1 prediction column for Numerai Classic.
        At least 1 prediction column and 1 ticker column for Numerai Signals.
        *args, **kwargs are passed to numerapi API.
        For example `version` argument in Numerai Classic submissions.
        """
        self.save_csv(dataf=dataf, file_name=file_name, cols=cols)
        self.upload_predictions(
            file_name=file_name, model_name=model_name,
            *args, **kwargs
        )

    def combine_csvs(self, csv_paths: list,
                     aux_cols: list,
                     era_col: str = None,
                     pred_col: str = 'prediction') -> pd.DataFrame:
        """
        Read in csv files and combine all predictions with a rank mean. \n
        Multi-target predictions will be averaged out. \n
        :param csv_paths: List of full paths to .csv prediction files. \n
        :param aux_cols: ['id'] for Numerai Classic. \n
        ['ticker', 'last_friday', 'data_type'], for example, with Numerai Signals. \n
        :param era_col: Column indicating era ('era' or 'last_friday'). \n
        Will be used for Grouping the rank mean if given. Skip groupby if no era_col provided. \n
        :param pred_col: 'prediction' for Numerai Classic and 'signal' for Numerai Signals.
        """
        all_datafs = [pd.read_csv(path, index_col=aux_cols) for path in tqdm(csv_paths)]
        final_dataf = pd.concat(all_datafs, axis="columns")
        # Remove issue of duplicate columns
        numeric_cols = final_dataf.select_dtypes(include=np.number).columns
        final_dataf.rename({k: str(v) for k, v in zip(numeric_cols, range(len(numeric_cols)))},
                           axis=1,
                           inplace=True)
        # Combine all numeric columns with rank mean
        num_dataf = final_dataf.select_dtypes(include=np.number)
        num_dataf = num_dataf.groupby(era_col) if era_col else num_dataf
        final_dataf[pred_col] = num_dataf.rank(pct=True, method="first").mean(axis=1)
        return final_dataf[[pred_col]]

    def _get_model_id(self, model_name: str) -> str:
        """
        Get ID needed for prediction uploading.
        :param model_name: Raw lowercase model name
        of Numerai model that you have access to.
        """
        return self.get_model_mapping[model_name]

    @property
    def get_model_mapping(self) -> dict:
        """Mapping between raw model names and model IDs."""
        return self.api.get_models()

    def _check_value_range(self, dataf: pd.DataFrame, cols: Union[str, list]):
        """ Check if all predictions are in range (0...1). """
        cols = [cols] if isinstance(cols, str) else cols
        for col in cols:
            if not dataf[col].between(0, 1).all():
                min_val, max_val = dataf[col].min(), dataf[col].max()
                raise ValueError(
                    f"Values must be between 0 and 1. \
Found min value of '{min_val}' and max value of '{max_val}' for column '{col}'."
                )

    def __call__(
            self,
            dataf: pd.DataFrame,
            model_name: str,
            file_name: str = "submission.csv",
            cols: Union[str, list] = "prediction",
            *args,
            **kwargs,
    ):
        """
        The most common use case will be to create a CSV and submit it immediately after that.
        full_submission handles this.
        """
        self.full_submission(
            dataf=dataf,
            file_name=file_name,
            model_name=model_name,
            cols=cols,
            *args,
            **kwargs,
        )

## 1. NumeraiClassicSubmitter

For Numerai Classic submissions. Uses [NumerAPI](https://numerapi.readthedocs.io/en/latest/_modules/numerapi/numerapi.html) under the hood.

Note that using submitters requires a `Key` object.

In [ ]:
#| export
class NumeraiClassicSubmitter(BaseSubmitter):
    """
    Submit for Numerai Classic.

    :param directory_path: Base directory to save and read prediction files from. \n
    :param key: Key object containing valid credentials for Numerai Classic. \n
    *args, **kwargs will be passed to NumerAPI initialization.
    """
    def __init__(self, directory_path: str, key: Key, *args, **kwargs):
        api = NumerAPI(public_id=key.pub_id, secret_key=key.secret_key, *args, **kwargs)
        super().__init__(
            directory_path=directory_path, api=api
        )

    def save_csv(
            self,
            dataf: pd.DataFrame,
            file_name: str = "submission.csv",
            cols: str = 'prediction',
            *args,
            **kwargs,
    ):
        """
        :param dataf: DataFrame which should have at least the following columns:
        1. id (as index column)
        2. cols (for example, 'prediction_mymodel'). Will be saved in 'prediction' column
        :param file_name: .csv file path.
        :param cols: Prediction column name.
        For example, 'prediction' or 'prediction_mymodel'.
        """
        sub_dataf = deepcopy(dataf)
        self._check_value_range(dataf=sub_dataf, cols=cols)

        full_path = str(self.dir / file_name)
        rich_print(
            f":page_facing_up: Saving predictions CSV to '{full_path}'. :page_facing_up:"
        )
        sub_dataf.loc[:, 'prediction'] = sub_dataf[cols]
        sub_dataf.loc[:, 'prediction'].to_csv(full_path, *args, **kwargs)

### Example usage 1: NumeraiClassicSubmitter

In [ ]:
# example 1
# Initialization (Random credentials)
test_dir = "test_sub"
classic_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
num_sub = NumeraiClassicSubmitter(directory_path=test_dir, key=classic_key)
assert num_sub.dir.is_dir()

# Create random dataframe
n_rows = 100
targets = "prediction_mymodel"
test_dataf = pd.DataFrame(np.random.uniform(size=n_rows), columns=[targets])
test_dataf["id"] = [uuid.uuid4() for _ in range(n_rows)]
test_dataf = test_dataf.set_index("id")
test_dataf.head(2)

No existing directory found at 'test_sub'. Creating directory...

,prediction_mymodel
id,
3c5a588a-a4e1-4b4d-976a-e86079ea5053,0.704015
d9ea0000-47a6-4e68-985f-36b3f9c51be4,0.667652


CSVs can be saved with `.save_csv`. `NumeraiClassicSubmitter` will automatically provide checks to make sure that data is saved correctly.

In [ ]:
file_name = "test.csv"
num_sub.save_csv(dataf=test_dataf, file_name=file_name, cols=targets)
num_sub.save_csv(dataf=test_dataf, file_name="test2.csv", cols=targets)
pd.read_csv(f"{test_dir}/{file_name}").head(2)

📄 Saving predictions CSV to 'test_sub/test.csv'. 📄

📄 Saving predictions CSV to 'test_sub/test2.csv'. 📄

,id,prediction
0,3c5a588a-a4e1-4b4d-976a-e86079ea5053,0.704015
1,d9ea0000-47a6-4e68-985f-36b3f9c51be4,0.667652


`NumeraiClassicSubmitter` also gives you the option to combine multiple predictions csvs that you already created. Prediction will be standardized by default.

In [ ]:
#| echo: false
show_doc(NumeraiClassicSubmitter.combine_csvs)

---

[source](https://github.com/crowdcent/numerblox/blob/master/numerblox/submission.py#L101){target="_blank" style="float:right; font-size:smaller"}

### BaseSubmitter.combine_csvs

>      BaseSubmitter.combine_csvs (csv_paths:list, aux_cols:list,
>                                  era_col:str=None, pred_col:str='prediction')

Read in csv files and combine all predictions with a rank mean. 

Multi-target predictions will be averaged out. 

:param csv_paths: List of full paths to .csv prediction files. 

:param aux_cols: ['id'] for Numerai Classic. 

['ticker', 'last_friday', 'data_type'], for example, with Numerai Signals. 

:param era_col: Column indicating era ('era' or 'last_friday'). 

Will be used for Grouping the rank mean if given. Skip groupby if no era_col provided. 

:param pred_col: 'prediction' for Numerai Classic and 'signal' for Numerai Signals.

In [ ]:
combined = num_sub.combine_csvs(["test_sub/test.csv", "test_sub/test2.csv"], aux_cols=['id'])
assert combined.columns == ['prediction']
combined.head(2)

  0%|          | 0/2 [00:00<?, ?it/s]

,prediction
id,
3c5a588a-a4e1-4b4d-976a-e86079ea5053,0.75
d9ea0000-47a6-4e68-985f-36b3f9c51be4,0.71


In [ ]:
#| include: false
def test_signal_validity(
        submitter: NumeraiClassicSubmitter, dataf: pd.DataFrame
):
    """ Test value range of prediction. """
    try:
        invalid_signal = deepcopy(dataf)
        invalid_signal.iloc[0]["prediction_mymodel"] += 10
        submitter.save_csv(
            invalid_signal,
            file_name="should_not_save.csv",
            cols="prediction_mymodel",
        )
    except ValueError:
        return True
    return False

assert test_signal_validity(num_sub, test_dataf)

Uncomment to save CSV and upload predictions in one go.

In [ ]:
# Full submission
# num_sub.full_submission(dataf=test_dataf, file_name='test.csv', cols=targets, model_name="test")

After a successful submission, contents can be removed to keep a clean environment.

In [ ]:
num_sub.remove_base_directory()
assert not os.path.exists(test_dir)

⚠ Deleting directory for 'NumeraiClassicSubmitter' ⚠
Path: '/home/clepelaars/numerblox/nbs/test_sub'

## 2. NumeraiSignalsSubmitter

Numerai Signals submissions. Uses [SignalsAPI](https://numerapi.readthedocs.io/en/latest/_modules/numerapi/signalsapi.html) under the hood.

In [ ]:
#| export
class NumeraiSignalsSubmitter(BaseSubmitter):
    """
    Submit for Numerai Signals.

    :param directory_path: Base directory to save and read prediction files from. \n
    :param key: Key object containing valid credentials for Numerai Signals. \n
    *args, **kwargs will be passed to SignalsAPI initialization.
    """

    def __init__(self, directory_path: str, key: Key, *args, **kwargs):
        api = SignalsAPI(
            public_id=key.pub_id, secret_key=key.secret_key, *args, **kwargs
        )
        super().__init__(
            directory_path=directory_path, api=api
        )
        self.supported_ticker_formats = [
            "cusip",
            "sedol",
            "ticker",
            "numerai_ticker",
            "bloomberg_ticker",
        ]

    def save_csv(
            self,
            dataf: pd.DataFrame,
            cols: list,
            file_name: str = "submission.csv",
            *args, **kwargs
    ):
        """
        :param dataf: DataFrame which should have at least the following columns:
         1. One of supported ticker formats (cusip, sedol, ticker, numerai_ticker or bloomberg_ticker)
         2. signal (Values between 0 and 1 (exclusive))
         Additional columns for if you include validation data (optional):
         3. friday_date (YYYYMMDD format date indication)
         4. data_type ('val' and 'live' partitions)

         :param cols: All cols that are saved in CSV.
         cols should contain at least 1 ticker column and a 'signal' column.
         For example: ['bloomberg_ticker', 'signal']
         :param file_name: .csv file path.
        """
        self._check_ticker_format(cols=cols)
        self._check_value_range(dataf=dataf, cols="signal")

        full_path = str(self.dir / file_name)
        rich_print(
            f":page_facing_up: Saving Signals predictions CSV to '{full_path}'. :page_facing_up:"
        )
        dataf.loc[:, cols].reset_index(drop=True).to_csv(
            full_path, index=False, *args, **kwargs
        )

    def _check_ticker_format(self, cols: list):
        """ Check for valid ticker format. """
        valid_tickers = set(cols).intersection(set(self.supported_ticker_formats))
        if not valid_tickers:
            raise NotImplementedError(
                f"No supported ticker format in {cols}). \
Supported: '{self.supported_ticker_formats}'"
            )

### Example usage 2: NumeraiSignalsSubmitter

Initialization (Random credentials)

In [ ]:
test_dir_signals = "test_sub_signals"
signals_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
signals_sub = NumeraiSignalsSubmitter(directory_path=test_dir_signals, key=signals_key)
assert signals_sub.dir.is_dir()

No existing directory found at 'test_sub_signals'. Creating directory...

In [ ]:
def create_random_signals_dataf(n_rows=5000):
    signals_test_dataf = pd.DataFrame(
        np.random.uniform(size=(n_rows, 1)), columns=["signal"]
    )
    signals_test_dataf["ticker"] = [
        "".join(choices(ascii_uppercase, k=4)) for _ in range(n_rows)
    ]
    last_friday = str((datetime.now() + relativedelta(weekday=FR(-1))).date()).replace("-", "")
    signals_test_dataf['last_friday'] = last_friday
    signals_test_dataf['data_type'] = 'live'
    return signals_test_dataf

signals_test_dataf = create_random_signals_dataf()
signals_test_dataf.head(2)

,signal,ticker,last_friday,data_type
0,0.907471,XONJ,20230901,live
1,0.149298,KDHF,20230901,live


In [ ]:
#| include: false
signals_cols = ["signal", "ticker", "data_type", "last_friday"]
file_name = "signals_test.csv"
signals_sub.save_csv(dataf=signals_test_dataf, file_name=file_name, cols=signals_cols)
signals_sub.save_csv(dataf=signals_test_dataf, file_name="signals_test2.csv", cols=signals_cols)
pd.read_csv(f"{test_dir_signals}/{file_name}").head(2)

📄 Saving Signals predictions CSV to 'test_sub_signals/signals_test.csv'. 📄

📄 Saving Signals predictions CSV to 'test_sub_signals/signals_test2.csv'. 📄

,signal,ticker,data_type,last_friday
0,0.907471,XONJ,live,20230901
1,0.149298,KDHF,live,20230901


In [ ]:
#| include: false
combined_signals = signals_sub.combine_csvs(csv_paths=["test_sub_signals/signals_test.csv",
                                               "test_sub_signals/signals_test2.csv"],
                                    aux_cols=['ticker', 'last_friday', 'data_type'],
                                    era_col='last_friday',
                                    pred_col='signal')
assert combined_signals.columns == ['signal']
combined_signals.head(2)

  0%|          | 0/2 [00:00<?, ?it/s]

,,,signal
ticker,last_friday,data_type,
XONJ,20230901,live,0.9088
KDHF,20230901,live,0.1594


Saving Signals CSV should fail if there is no valid ticker column or if `signal` has values outside the range $[0...1]$.

In [ ]:
#| include: false
def test_signal_validity(
    submitter: NumeraiSignalsSubmitter, signals_dataf: pd.DataFrame
):
    """ Test value range of signal. """
    try:
        invalid_signal = deepcopy(signals_dataf)
        invalid_signal.loc[0, "signal"] += 10
        submitter.save_csv(
            invalid_signal,
            file_name="should_not_save.csv",
            cols=list(invalid_signal.columns),
        )
    except ValueError:
        return True
    return False


def test_ticker_validity(
    submitter: NumeraiSignalsSubmitter, signals_dataf: pd.DataFrame
):
    """ Test safeguard if ticker column is not valid. """
    try:
        invalid_ticker = deepcopy(signals_dataf)
        invalid_ticker = invalid_ticker.rename(
            {"ticker": "not_a_valid_ticker_format"}, axis=1
        )
        submitter.save_csv(
            invalid_ticker,
            file_name="should_not_save.csv",
            cols=list(invalid_ticker.columns),
        )
    except NotImplementedError:
        return True
    return False

assert test_signal_validity(signals_sub, signals_test_dataf)
assert test_ticker_validity(signals_sub, signals_test_dataf)

Uncomment to save CSV and upload predictions in one go.

In [ ]:
# Full Signals submission
# signals_sub.full_submission(dataf=signals_test_dataf, file_name='signals_test.csv', cols=signals_cols, model_name="test")

After a successful submission, contents can be removed to keep a clean environment.

In [ ]:
signals_sub.remove_base_directory()
assert not os.path.exists(test_dir_signals)

⚠ Deleting directory for 'NumeraiSignalsSubmitter' ⚠
Path: '/home/clepelaars/numerblox/nbs/test_sub_signals'

## 3. NumerBaySubmitter

Wrapper on top of the tournament submitters, submits to [NumerBay](https://numerbay.ai/) to fulfill sale orders. Make sure you have numerbay installed (`pip install numerbay`).

In [ ]:
#| export
class NumerBaySubmitter(BaseSubmitter):
    """
    Submit to NumerBay to fulfill sale orders, in addition to submission to Numerai.

    :param tournament_submitter: Base tournament submitter (NumeraiClassicSubmitter or NumeraiSignalsSubmitter). This submitter will use the same directory path.
    :param upload_to_numerai: Whether to also submit to Numerai using the tournament submitter. Defaults to True, set to False to only upload to NumerBay.
    :param numerbay_username: NumerBay username
    :param numerbay_password: NumerBay password
    """
    def __init__(self,
                 tournament_submitter: Union[NumeraiClassicSubmitter, NumeraiSignalsSubmitter],
                 upload_to_numerai: bool = True,
                 numerbay_username: str = None,
                 numerbay_password: str = None):
        super().__init__(
            directory_path=str(tournament_submitter.dir), api=tournament_submitter.api
        )
        from numerbay import NumerBay
        self.numerbay_api = NumerBay(username=numerbay_username, password=numerbay_password)
        self.tournament_submitter = tournament_submitter
        self.upload_to_numerai = upload_to_numerai

    def upload_predictions(self,
                           file_name: str,
                           model_name: str,
                           numerbay_product_full_name: str,
                           *args,
                           **kwargs):
        """
        Upload CSV file to NumerBay (and Numerai if 'upload_to_numerai' is True) for given model name and NumerBay product full name.
        :param file_name: File name/path relative to directory_path.
        :param model_name: Lowercase raw model name (For example, 'integration_test').
        :param numerbay_product_full_name: NumerBay product full name in the format of [category]-[product name], e.g. 'numerai-predictions-numerbay'
        """
        if self.upload_to_numerai:
            self.tournament_submitter.upload_predictions(file_name, model_name, *args, **kwargs)


        full_path = str(self.dir / file_name)
        api_type = str(self.numerbay_api.__class__.__name__)
        rich_print(
            f":airplane: {api_type}: Uploading predictions from '{full_path}' for NumerBay product [bold blue]'{numerbay_product_full_name}'[/bold blue] :airplane:"
        )
        artifact = self.numerbay_api.upload_artifact(
            str(full_path), product_full_name=numerbay_product_full_name
        )
        if artifact:
            rich_print(
                f":thumbs_up: {api_type} submission of '{full_path}' for NumerBay product [bold blue]{numerbay_product_full_name}[/bold blue] is successful! :thumbs_up:"
            )
        else:
            rich_print(f":warning: [red] Upload skipped for NumerBay product '{numerbay_product_full_name}', the product uses buyer-side encryption "
                        "but does not have any active sale order to upload for. [/red] :warning:")

    def full_submission(
        self,
        dataf: pd.DataFrame,
        model_name: str,
        cols: Union[str, list],
        numerbay_product_full_name: str,
        file_name: str = 'submission.csv',
        *args,
        **kwargs,
    ):
        """
        Save DataFrame to csv and upload predictions through API.

        :param dataf: Main DataFrame containing `cols`.
        :param model_name: Lowercase Numerai model name.
        :param numerbay_product_full_name: NumerBay product full name in the format of [category]-[product name], e.g. 'numerai-predictions-numerbay'
        :param file_name: path to save model to relative to base directory.
        :param cols: Columns to be saved in submission file.
        1 prediction column for Numerai Classic.
        At least 1 prediction column and 1 ticker column for Numerai Signals.
        *args, **kwargs are passed to numerapi API.
        For example `version` argument in Numerai Classic submissions.
        """
        self.save_csv(dataf=dataf, file_name=file_name, cols=cols)
        self.upload_predictions(
            file_name=file_name, model_name=model_name, numerbay_product_full_name=numerbay_product_full_name,
            *args, **kwargs
        )

    def combine_csvs(self, *args,**kwargs) -> pd.DataFrame:
        return self.tournament_submitter.combine_csvs(*args,**kwargs)

    def save_csv(self, *args, **kwargs):
        self.tournament_submitter.save_csv(*args, **kwargs)

    @property
    def get_model_mapping(self) -> dict:
        return self.tournament_submitter.api.get_models()

    def __call__(
            self,
            dataf: pd.DataFrame,
            model_name: str,
            numerbay_product_full_name: str,
            file_name: str = "submission.csv",
            cols: Union[str, list] = "prediction",
            *args,
            **kwargs,
    ):
        """
        The most common use case will be to create a CSV and submit it immediately after that.
        full_submission handles this.
        """
        self.full_submission(
            dataf=dataf,
            file_name=file_name,
            model_name=model_name,
            numerbay_product_full_name=numerbay_product_full_name,
            cols=cols,
            *args,
            **kwargs,
        )

### Example usage 3: NumerBaySubmitter

Numerai submissions (existing file)

In [ ]:
# numerai_submitter = NumeraiClassicSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.upload_predictions(file_name='upload.csv', model_name='mymodel',
#                                 numerbay_product_full_name='numerai-predictions-myproduct')

Numerai submissions (from NumerFrame)

In [ ]:
# numerai_submitter = NumeraiClassicSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.full_submission(dataf, file_name='upload.csv', model_name='mymodel',
#                              numerbay_product_full_name='numerai-predictions-myproduct',
#                              cols='some_pred_col')

Signals submissions (existing file)

In [ ]:
# signals_submitter = NumeraiSignalsSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=signals_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.upload_predictions(file_name='upload-signals.csv', model_name='mymodel',
#                                 numerbay_product_full_name='signals-predictions-myproduct')

Signals submissions (from NumerFrame)

In [ ]:
# signals_submitter = NumeraiSignalsSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=signals_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.full_submission(dataf, file_name='upload-signals.csv', model_name='mymodel',
#                              numerbay_product_full_name='signals-predictions-myproduct',
#                              cols=['bloomberg_ticker', 'friday_date', 'data_type', 'signal'])

------------------------------------------------------------